In [1]:
import pandas as pd
from tensorflow.keras import models
from tensorflow.keras import layers
import numpy as np
from collections import Counter
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM, Flatten, Reshape, MaxPool1D, AveragePooling1D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard 
import pandas as pd
import numpy as np
import os
import datetime 
from matplotlib import pyplot as plt

In [29]:
train_data = pd.read_csv("train_pos.txt", header = None, sep=" ", names = ['word', 'pos'], dtype = str)
test_data = pd.read_csv("test_pos.txt", header = None, sep=" ", names = ['word', 'pos'],dtype=str)
test_data.head()

,word,pos
0,Rockwell,NNP
1,International,NNP
2,Corp.,NNP
3,'s,POS
4,Tulsa,NNP


In [30]:
s = set("".join(str(train_data['word'])))
s.remove("\n")
s.remove(" ")
# s.remove(",")
# s.remove(".")
unique = list(set(s))
unique.sort()
vocab = dict(zip(unique, range(1,len(unique)+1)))
VOCAB_SIZE = len(vocab)
vocab

{',': 1,
 '.': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '8': 10,
 '9': 11,
 ':': 12,
 'C': 13,
 'F': 14,
 'L': 15,
 'N': 16,
 'S': 17,
 'a': 18,
 'b': 19,
 'c': 20,
 'd': 21,
 'e': 22,
 'f': 23,
 'g': 24,
 'h': 25,
 'i': 26,
 'j': 27,
 'm': 28,
 'n': 29,
 'o': 30,
 'p': 31,
 'r': 32,
 's': 33,
 't': 34,
 'u': 35,
 'w': 36,
 'y': 37}

In [76]:
def encode(name, maxsize):
  encoding = []
  if name is None: return name
  for c in name:
    if c in vocab:
      encoding.append(vocab[c])
  if len(encoding) < maxsize:
    for i in range(maxsize-len(encoding)):
      encoding.append(0)
  
  return encoding
 
 
def preprocess(data, col = 'word', maxsize = 43):
  new_index = data[col].str.len().sort_values().index
  data = data.reindex(new_index)
  # maxsize = len(data[col].dropna().iloc[-1])

  data[col] = data[col].apply(lambda  x: encode(str(x), maxsize = maxsize))
  return data.dropna()
 
# train_data['word'] = train_data['word'].dropna()


# print(MAX_WORD_SIZE)
train_data_pre = preprocess(train_data)
MAX_WORD_SIZE = len(train_data_pre.iloc[-1,0])
MAX_WORD_SIZE = 43

train_data_pre

,word,pos
220661,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",.
199532,"[18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",DT
75209,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",","
75212,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",","
199524,"[18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",DT
...,...,...
124574,"[18, 26, 29, 34, 26, 34, 24, 32, 22, 18, 34, 3...",JJ
146350,"[20, 30, 28, 31, 35, 34, 22, 32, 26, 29, 34, 2...",JJ
146209,"[20, 30, 28, 31, 35, 34, 22, 32, 26, 29, 34, 2...",JJ
146095,"[20, 30, 28, 31, 35, 34, 22, 32, 26, 29, 34, 2...",JJ


In [77]:
test_data_pre = preprocess(test_data)
# MAX_WORD_SIZE_TEST = len(test_data_pre['word'].dropna().iloc[-1])
test_data_pre

,word,pos
49387,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",.
9741,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",","
44013,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",CC
9747,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",.
44015,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",","
...,...,...
37899,"[22, 33, 33, 22, 32, 21, 22, 22, 30, 31, 22, 2...",JJ
32650,"[26, 29, 34, 22, 32, 29, 18, 34, 26, 30, 29, 1...",NNS
31879,"[36, 22, 32, 22, 18, 26, 29, 34, 25, 26, 33, 3...",JJ
9180,"[31, 32, 22, 33, 26, 21, 22, 29, 34, 29, 18, 3...",NN


In [79]:
train_x = np.array([np.array(item) for item in train_data_pre['word']])
test_x = np.array([np.array(item) for item in test_data_pre['word']])
train_x.shape, test_x.shape


((211727, 43), (47377, 43))

In [81]:
POS_TAGS_DICT = {}
for index, item in enumerate(set(train_data['pos'].dropna()),0):
  POS_TAGS_DICT[item] = index
def encode_labels(labels):
  return np.array([POS_TAGS_DICT[item] for item in labels])

def embed_one_hot(data_series, VOCAB_SIZE): #, labels = None
  X = []
  Y = []
  l = []
  def fill_position(position, size):
    t = np.zeros(size)
    if position < len(t):
      t[position] = 1
    return t
 
  for item in data_series:
    l = fill_position(item, VOCAB_SIZE)
    l = np.array(l)
    X.append(l)
  X = np.array(X)
  return X


POS_TAGS = len(train_data['pos'].dropna().unique())


train_y = encode_labels(train_data_pre['pos'])
train_y_embed = embed_one_hot(train_y, POS_TAGS)
test_y = encode_labels(test_data_pre['pos'])
test_y_embed = embed_one_hot(test_y, POS_TAGS)



train_y_embed

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [82]:
MAX_WORD_SIZE, POS_TAGS, POS_TAGS_DICT

(43,
 44,
 {'#': 38,
  '$': 5,
  "''": 25,
  '(': 1,
  ')': 3,
  ',': 16,
  '.': 13,
  ':': 23,
  'CC': 40,
  'CD': 22,
  'DT': 14,
  'EX': 31,
  'FW': 34,
  'IN': 29,
  'JJ': 24,
  'JJR': 0,
  'JJS': 35,
  'MD': 2,
  'NN': 27,
  'NNP': 10,
  'NNPS': 28,
  'NNS': 39,
  'PDT': 32,
  'POS': 36,
  'PRP': 6,
  'PRP$': 8,
  'RB': 17,
  'RBR': 19,
  'RBS': 30,
  'RP': 21,
  'SYM': 9,
  'TO': 33,
  'UH': 12,
  'VB': 4,
  'VBD': 20,
  'VBG': 11,
  'VBN': 43,
  'VBP': 7,
  'VBZ': 18,
  'WDT': 15,
  'WP': 26,
  'WP$': 37,
  'WRB': 41,
  '``': 42})

In [83]:


# TRAINING CONFIGS
callback = EarlyStopping(monitor='val_accuracy', patience=5,restore_best_weights=True)
EPOCHS = 5
HIDDEN_UNITS = 100
# DROPOUT = 0.2
EMBEDDING_SIZE = 25
# OUTPUT_SIZE = 1
# OPTIM = Adam(learning_rate=0.001)
# VALIDATION_SPLIT = 0.2
# METRICS = ['accuracy']


model = models.Sequential()
model.add(layers.Embedding(input_dim=VOCAB_SIZE+1, output_dim=EMBEDDING_SIZE, input_length=MAX_WORD_SIZE))
model.add(layers.Convolution1D(filters=HIDDEN_UNITS, kernel_size= 3, activation="relu"))
model.add(MaxPool1D(2))
# model.add(layers.SimpleRNN(5))
model.add(layers.Flatten())
model.add(layers.Dense(POS_TAGS, activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics = ['accuracy'])
print(model.summary())
print(model.output_shape)
history = model.fit(train_x, train_y_embed, epochs=EPOCHS)#, validation_split = VALIDATION_SPLIT,callbacks=[callback])


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 43, 25)            950       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 41, 100)           7600      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 20, 100)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 44)                88044     
Total params: 96,594
Trainable params: 96,594
Non-trainable params: 0
_________________________________________________________________
None
(None, 44)
Epoch 1/5
6617/6617 [==============================] - 43s 6ms/step - loss: 1.2558 - accuracy: 0.626

In [84]:
test_y_pred = model.predict(test_x)
train_y_pred = model.predict(train_x)
# test_y_pred[:,0]

In [85]:
train_y_pred_max = train_y_pred.argmax(axis = 1)

In [1]:
# from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
# import seaborn as sn

# # cm = confusion_matrix(train_y, train_y_pred,normalize='true')

# # df_cm = pd.DataFrame(cm, index = [i for i in POS_TAGS_DICT],
# #                   columns = [i for i in POS_TAGS_DICT])
# # plt.figure(figsize = (10,7))
# # sn.heatmap(df_cm, annot=True)
# plot_confusion_matrix(model, train_x, train_y, normalize='true')

In [88]:
model.summary()


Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_34 (Embedding)     (None, 43, 25)            1000      
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 42, 100)           5100      
_________________________________________________________________
max_pooling1d_32 (MaxPooling (None, 21, 100)           0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 2100)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 45)                94545     
Total params: 100,645
Trainable params: 100,645
Non-trainable params: 0
_________________________________________________________________


In [86]:
sentences_end_points = np.where(train_data['word'].str.contains('\n'))[0]
sentences_end_points

array([    37,     65,     95, ..., 220615, 220635, 220662])

In [101]:
ll = []
for index, item in enumerate(sentences_end_points, 0):
  if index == 0:
    ll.append(list(train_data['pos'][0:item]))
  else:
    ll.append(list(train_data['pos'][sentences_end_points[index-1]+1:item]))

ll[1]

['NNP',
 'IN',
 'DT',
 'NNP',
 'NNP',
 'NNP',
 'POS',
 'VBN',
 'NN',
 'TO',
 'DT',
 'NN',
 'JJ',
 'NN',
 'VBZ',
 'VBN',
 'TO',
 'VB',
 'DT',
 'NN',
 'IN',
 'NN',
 'IN',
 'DT',
 'JJ',
 'NN',
 '.']

In [103]:
POS_TAGS_DICT

{'#': 38,
 '$': 5,
 "''": 25,
 '(': 1,
 ')': 3,
 ',': 16,
 '.': 13,
 ':': 23,
 'CC': 40,
 'CD': 22,
 'DT': 14,
 'EX': 31,
 'FW': 34,
 'IN': 29,
 'JJ': 24,
 'JJR': 0,
 'JJS': 35,
 'MD': 2,
 'NN': 27,
 'NNP': 10,
 'NNPS': 28,
 'NNS': 39,
 'PDT': 32,
 'POS': 36,
 'PRP': 6,
 'PRP$': 8,
 'RB': 17,
 'RBR': 19,
 'RBS': 30,
 'RP': 21,
 'SYM': 9,
 'TO': 33,
 'UH': 12,
 'VB': 4,
 'VBD': 20,
 'VBG': 11,
 'VBN': 43,
 'VBP': 7,
 'VBZ': 18,
 'WDT': 15,
 'WP': 26,
 'WP$': 37,
 'WRB': 41,
 '``': 42}

In [112]:
def transition_matrix(transitions):
    n = 44 #number of states

    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

#test:

# t = [1,1,2,6,8,5,5,7,8,8,1,1,4,5,5,0,0,0,1,1,4,4,5,1,3,3,4,5,4,1,1]
sums = np.zeros((POS_TAGS, POS_TAGS))
for item in ll:
  t = [POS_TAGS_DICT[i] for i in item]

  m = transition_matrix(t)
  # print(len(m))
  sums += np.array(m)
# for row in m: print(' '.join('{0:.2f}'.format(x) for x in row))
sums.shape

(44, 44)

In [143]:
transition_table = np.array([sums[i]/np.sum(sums[i]) for i in range(len(sums))])
transition_table.shape

(44, 44)

In [142]:
emission_table = train_y_pred
emission_table.shape

(211727, 44)

In [162]:
train_data['pos'] = train_data['pos'].dropna()
pi = []
for k, v in Counter(train_data['pos'].dropna()).items():
  pi.append(v/len(train_data['pos'].dropna()))

initial_table = np.array(pi)
initial_table

array([1.42386186e-01, 1.07515810e-01, 8.65973636e-02, 2.19527977e-02,
       3.12052785e-02, 2.24959500e-02, 2.39978841e-02, 2.84186712e-02,
       6.18012818e-02, 6.43233976e-02, 9.39133885e-02, 5.08673906e-02,
       2.53722955e-02, 8.35509878e-03, 4.16904788e-02, 1.35457452e-02,
       1.54538628e-02, 8.88408186e-03, 3.92722704e-02, 7.23100974e-03,
       7.05153334e-03, 3.18570612e-02, 9.72951017e-04, 1.02348779e-02,
       1.70030275e-04, 1.29411931e-03, 8.26536058e-03, 1.32718076e-03,
       1.98368654e-03, 1.80421014e-02, 1.76642563e-03, 2.49850043e-03,
       1.51610328e-03, 4.02877290e-03, 4.51052535e-03, 2.25762420e-03,
       9.02105069e-04, 2.59768475e-04, 3.92014245e-04, 4.94504716e-03,
       1.79476401e-04, 1.65307212e-04, 2.83383791e-05, 7.08459478e-05])

In [156]:
from typing import List, Optional, Tuple

def step(mu_prev: np.ndarray,
         emission_probs: np.ndarray,
         transition_probs: np.ndarray,
         observed_state: int) -> Tuple[np.ndarray, np.ndarray]:
    """Runs one step of the Viterbi algorithm.
    
    Args:
        mu_prev: probability distribution with shape (num_hidden),
            the previous mu
        emission_probs: the emission probability matrix (num_hidden,
            num_observed)
        transition_probs: the transition probability matrix, with
            shape (num_hidden, num_hidden)
        observed_state: the observed state at the current step
    
    Returns:
        - the mu for the next step
        - the maximizing previous state, before the current state,
          as an int array with shape (num_hidden)
    """
    
    pre_max = mu_prev * transition_probs.T
    max_prev_states = np.argmax(pre_max, axis=1)
    max_vals = pre_max[np.arange(len(max_prev_states)), max_prev_states]
    mu_new = max_vals * emission_probs[:, observed_state]
    
    return mu_new, max_prev_states


def viterbi(emission_probs: np.ndarray,
            transition_probs: np.ndarray,
            start_probs: np.ndarray,
            observed_states: List[int]) -> Tuple[List[int], float]:
    """Runs the Viterbi algorithm to get the most likely state sequence.
    
    Args:
        emission_probs: the emission probability matrix (num_hidden,
            num_observed)
        transition_probs: the transition probability matrix, with
            shape (num_hidden, num_hidden)
        start_probs: the initial probabilies for each state, with shape
            (num_hidden)
        observed_states: the observed states at each step
    
    Returns:
        - the most likely series of states
        - the joint probability of that series of states and the observed
    """
    
    # Runs the forward pass, storing the most likely previous state.
    mu = start_probs * emission_probs[:, observed_states[0]]
    all_prev_states = []
    for observed_state in observed_states[1:]:
        mu, prevs = step(mu, emission_probs, transition_probs, observed_state)
        all_prev_states.append(prevs)
    
    # Traces backwards to get the maximum likelihood sequence.
    state = np.argmax(mu)
    sequence_prob = mu[state]
    state_sequence = [state]
    for prev_states in all_prev_states[::-1]:
        state = prev_states[state]
        state_sequence.append(state)
    
    return state_sequence[::-1], sequence_prob

In [164]:
max_seq, seq_prob = viterbi(
    emission_table.T,
    transition_table,
    initial_table,
    [4,1,24,2],
)

In [166]:
max_seq

[14, 10, 10, 16]